In [1]:
#importing the libraries
import numpy as np
import pandas as pd
from scipy import stats

In [2]:
#loading Dataset and picking only useful columns

df_lan = pd.read_excel("../Dataset/Input_Dataset/DDW-C18-0000.xlsx", header = [1])
df_lan = df_lan[(df_lan.iloc[:,4] == "Total") & (df_lan.iloc[:, 3] != "Total")].iloc[:, [0, 2, 3, 5, 8]]
df_lan.columns = ["State Code", "Name", "Category", "two lang", "three lang"]
df_lan.head()

,State Code,Name,Category,two lang,three lang
14,00,INDIA,Rural,162641485,35383989
24,00,INDIA,Urban,152347285,50625591
44,01,JAMMU & KASHMIR,Rural,4167238,1258724
54,01,JAMMU & KASHMIR,Urban,2015952,837496
74,02,HIMACHAL PRADESH,Rural,981518,280817


In [3]:
#Finding the unique states
unique_st = list(df_lan["State Code"].unique())

In [4]:
#Creating the new dict
dict = {"state-code": [],
        "Name": [], 
        "Rural two lang" : [],
        "Rural three lang" : [], 
        "Urban two lang" : [],
        "Urban three lang" : []}

In [5]:
#transposing the dataset to new dataset
for st in unique_st:
    dict["state-code"].append(st)
    dict["Name"].append(df_lan[df_lan["State Code"] == st]["Name"].iloc[0])
    dict["Rural two lang"].append(df_lan[(df_lan["State Code"] == st) & (df_lan["Category"] == "Rural")]["two lang"].iloc[0])
    dict["Rural three lang"].append(df_lan[(df_lan["State Code"] == st) & (df_lan["Category"] == "Rural")]["three lang"].iloc[0])
    dict["Urban two lang"].append(df_lan[(df_lan["State Code"] == st) & (df_lan["Category"] == "Urban")]["two lang"].iloc[0])
    dict["Urban three lang"].append(df_lan[(df_lan["State Code"] == st) & (df_lan["Category"] == "Urban")]["three lang"].iloc[0])

In [6]:
#Creating new dataset
df_lan_new = pd.DataFrame(dict)

In [7]:
df_lan_new.head()

,state-code,Name,Rural two lang,Rural three lang,Urban two lang,Urban three lang
0,00,INDIA,162641485,35383989,152347285,50625591
1,01,JAMMU & KASHMIR,4167238,1258724,2015952,837496
2,02,HIMACHAL PRADESH,981518,280817,261187,66469
3,03,PUNJAB,7083180,4230870,5952044,3599043
4,04,CHANDIGARH,11723,4353,568197,317626


In [8]:
#loading Dataset and picking only useful columns
df_cen = pd.read_excel("../Dataset/Input_Dataset/DDW_PCA0000_2011_Indiastatedist.xlsx")
df_cen = df_cen[((df_cen["TRU"] != "Total") & ((df_cen["Level"] == "India") | (df_cen["Level"] == "STATE")))].iloc[:, [7, 8, 10]]
df_cen["Name"] = df_cen["Name"].apply(lambda x : x.upper())
df_cen.head()

,Name,TRU,TOT_P
1,INDIA,Rural,833748852
2,INDIA,Urban,377106125
4,JAMMU & KASHMIR,Rural,9108060
5,JAMMU & KASHMIR,Urban,3433242
73,HIMACHAL PRADESH,Rural,6176050


In [9]:
#Finding the unique states
unique_st = list(df_cen["Name"].unique())

In [10]:
#Creating the new dict
dict = {"Name": [], 
        "TOT_R" : [],
        "TOT_U" : []}

In [11]:
#transposing the dataset to new dataset
for st in unique_st:
    dict["Name"].append(st)
    dict["TOT_R"].append(df_cen[(df_cen["Name"] == st) & (df_cen["TRU"] == "Rural")]["TOT_P"].iloc[0])
    dict["TOT_U"].append(df_cen[(df_cen["Name"] == st) & (df_cen["TRU"] == "Urban")]["TOT_P"].iloc[0])

In [12]:
#Creating new dataset
df_cen_new = pd.DataFrame(dict)
df_cen_new.head()

,Name,TOT_R,TOT_U
0,INDIA,833748852,377106125
1,JAMMU & KASHMIR,9108060,3433242
2,HIMACHAL PRADESH,6176050,688552
3,PUNJAB,17344192,10399146
4,CHANDIGARH,28991,1026459


In [13]:
#Merging df_lan and df_cen 
df_final = pd.merge(df_lan_new, df_cen_new, on = "Name")

#Calculating the number of population of rural and urban speaking exactly one language
df_final["Rural Exact one lang"] = df_final["TOT_R"] - df_final["Rural two lang"]
df_final["Urban Exact one lang"] = df_final["TOT_U"] - df_final["Urban two lang"]
#Calculating the number of population of rural and urban speaking exactly two language
df_final["Rural Exact two lang"] = df_final["Rural two lang"] - df_final["Rural three lang"]
df_final["Urban Exact two lang"] = df_final["Urban two lang"] - df_final["Urban three lang"]

#Renaming the column name
df_final = df_final.rename(columns={"Rural three lang" : "Rural Exact three lang", "Urban three lang" : "Urban Exact three lang"})
df_final.drop(["Rural two lang", "Urban two lang"], axis = 1, inplace=True)
df_final.head()

,state-code,Name,Rural Exact three lang,Urban Exact three lang,TOT_R,TOT_U,Rural Exact one lang,Urban Exact one lang,Rural Exact two lang,Urban Exact two lang
0,00,INDIA,35383989,50625591,833748852,377106125,671107367,224758840,127257496,101721694
1,01,JAMMU & KASHMIR,1258724,837496,9108060,3433242,4940822,1417290,2908514,1178456
2,02,HIMACHAL PRADESH,280817,66469,6176050,688552,5194532,427365,700701,194718
3,03,PUNJAB,4230870,3599043,17344192,10399146,10261012,4447102,2852310,2353001
4,04,CHANDIGARH,4353,317626,28991,1026459,17268,458262,7370,250571


In [14]:
#Calculating the ratio
df_final["Ratio_1"] = df_final["Urban Exact one lang"] / df_final["Rural Exact one lang"]
df_final["Ratio_2"] = df_final["Urban Exact two lang"] / df_final["Rural Exact two lang"]
df_final["Ratio_3"] = df_final["Urban Exact three lang"] / df_final["Rural Exact three lang"]
df_final["Ratio"] = df_final["TOT_U"] / df_final["TOT_R"]

In [15]:
df_final.head()

,state-code,Name,Rural Exact three lang,Urban Exact three lang,TOT_R,TOT_U,Rural Exact one lang,Urban Exact one lang,Rural Exact two lang,Urban Exact two lang,Ratio_1,Ratio_2,Ratio_3,Ratio
0,00,INDIA,35383989,50625591,833748852,377106125,671107367,224758840,127257496,101721694,0.334907,0.799338,1.430749,0.452302
1,01,JAMMU & KASHMIR,1258724,837496,9108060,3433242,4940822,1417290,2908514,1178456,0.286853,0.405175,0.665353,0.376945
2,02,HIMACHAL PRADESH,280817,66469,6176050,688552,5194532,427365,700701,194718,0.082272,0.277890,0.236699,0.111487
3,03,PUNJAB,4230870,3599043,17344192,10399146,10261012,4447102,2852310,2353001,0.433398,0.824946,0.850663,0.599575
4,04,CHANDIGARH,4353,317626,28991,1026459,17268,458262,7370,250571,26.538221,33.998779,72.967149,35.406126


In [16]:
#Calculating the p-values
df_final["p-value"] = df_final.apply(lambda row : stats.ttest_1samp([row.Ratio_1, row.Ratio_2, row.Ratio_3], popmean = row.Ratio)[1], axis = 1)

In [17]:
df_final.head()

,state-code,Name,Rural Exact three lang,Urban Exact three lang,TOT_R,TOT_U,Rural Exact one lang,Urban Exact one lang,Rural Exact two lang,Urban Exact two lang,Ratio_1,Ratio_2,Ratio_3,Ratio,p-value
0,00,INDIA,35383989,50625591,833748852,377106125,671107367,224758840,127257496,101721694,0.334907,0.799338,1.430749,0.452302,0.332407
1,01,JAMMU & KASHMIR,1258724,837496,9108060,3433242,4940822,1417290,2908514,1178456,0.286853,0.405175,0.665353,0.376945,0.568997
2,02,HIMACHAL PRADESH,280817,66469,6176050,688552,5194532,427365,700701,194718,0.082272,0.277890,0.236699,0.111487,0.279581
3,03,PUNJAB,4230870,3599043,17344192,10399146,10261012,4447102,2852310,2353001,0.433398,0.824946,0.850663,0.599575,0.523689
4,04,CHANDIGARH,4353,317626,28991,1026459,17268,458262,7370,250571,26.538221,33.998779,72.967149,35.406126,0.592084


In [18]:
#Extracting only useful columns
geography_india_1 = df_final.loc[:, ["state-code", "Rural Exact one lang", "Urban Exact one lang", "TOT_R", "TOT_U", "p-value"]]

#Calculating the rural and urban percentage
geography_india_1["urban-percentage"] = geography_india_1["Urban Exact one lang"] * 100 / geography_india_1["TOT_U"]
geography_india_1["rural-percentage"] = geography_india_1["Rural Exact one lang"] * 100 / geography_india_1["TOT_R"]

#Droping the unimportant columns
geography_india_1.drop(["Rural Exact one lang", "Urban Exact one lang", "TOT_R", "TOT_U"], axis = 1, inplace = True)
geography_india_1 = geography_india_1.rename(columns = {"State Code" : "state-code"})

#Renaming the columns name and reordering them
geography_india_1 = geography_india_1[["state-code", "urban-percentage", "rural-percentage", "p-value"]]
geography_india_1.head()

,state-code,urban-percentage,rural-percentage,p-value
0,00,59.600952,80.492749,0.332407
1,01,41.281389,54.246700,0.568997
2,02,62.067208,84.107674,0.279581
3,03,42.764108,59.161084,0.523689
4,04,44.644940,59.563313,0.592084


In [19]:
#Extracting only useful columns
geography_india_2 = df_final.loc[:, ["state-code", "Rural Exact two lang", "Urban Exact two lang", "TOT_R", "TOT_U", "p-value"]]

#Calculating the rural and urban percentage
geography_india_2["urban-percentage"] = geography_india_2["Urban Exact two lang"] * 100 / geography_india_2["TOT_U"]
geography_india_2["rural-percentage"] = geography_india_2["Rural Exact two lang"] * 100 / geography_india_2["TOT_R"]

#Droping the unimportant columns
geography_india_2.drop(["Rural Exact two lang", "Urban Exact two lang", "TOT_R", "TOT_U"], axis = 1, inplace = True)

#Renaming the columns name and reordering them
geography_india_2 = geography_india_2.rename(columns = {"State Code" : "state-code"})
geography_india_2 = geography_india_2[["state-code", "urban-percentage", "rural-percentage", "p-value"]]
geography_india_2.head()

,state-code,urban-percentage,rural-percentage,p-value
0,00,26.974288,15.263289,0.332407
1,01,34.324874,31.933408,0.568997
2,02,28.279346,11.345455,0.279581
3,03,22.626868,16.445332,0.523689
4,04,24.411204,25.421683,0.592084


In [20]:
#Extracting only useful columns
geography_india_3 = df_final.loc[:, ["state-code", "Rural Exact three lang", "Urban Exact three lang", "TOT_R", "TOT_U", "p-value"]]

#Calculating the rural and urban percentage
geography_india_3["urban-percentage"] = geography_india_3["Urban Exact three lang"] * 100 / geography_india_3["TOT_U"]
geography_india_3["rural-percentage"] = geography_india_3["Rural Exact three lang"] * 100 / geography_india_3["TOT_R"]

#Droping the unimportant columns
geography_india_3.drop(["Rural Exact three lang", "Urban Exact three lang", "TOT_R", "TOT_U"], axis = 1, inplace = True)

#Renaming the columns name and reordering them
geography_india_3 = geography_india_3.rename(columns = {"State Code" : "state-code"})
geography_india_3 = geography_india_3[["state-code", "urban-percentage", "rural-percentage", "p-value"]]
geography_india_3.head()

,state-code,urban-percentage,rural-percentage,p-value
0,00,13.424760,4.243963,0.332407
1,01,24.393736,13.819891,0.568997
2,02,9.653447,4.546871,0.279581
3,03,34.609025,24.393584,0.523689
4,04,30.943857,15.015005,0.592084


In [21]:
#Storing the final result
geography_india_1.to_csv("../Dataset/Output_Dataset/geography-india-a.csv", index = False)
geography_india_2.to_csv("../Dataset/Output_Dataset/geography-india-b.csv", index = False)
geography_india_3.to_csv("../Dataset/Output_Dataset/geography-india-c.csv", index = False)